In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
import seaborn as sns
from matplotlib import pyplot as plt
from geopy import distance

In [2]:
# combine type and model
def combine_type_model(df_in, train_set):
    df_cleaned = df_in.copy()
    condo_type_all=['condominium','apartment','executive condominium','walk-up apt']
    k=df_cleaned [df_cleaned ['model'].isna()]
    indices_model =k.index
    # Fill missing values by ‘name’ using majority vote. If there is no same community, fill in by ‘type’.
    for i in indices_model:
        condo_type,condo_model,condo_name=df_cleaned.loc[i]['type'],df_cleaned.loc[i]['model'],df_cleaned.loc[i]['name']
        df_filter =df_cleaned[(df_cleaned ['model']==df_cleaned ['type'])&(df_cleaned['name'] == condo_name)]
        df_filter_dropdup=df_filter.drop_duplicates('model')
        if  df_filter_dropdup.shape[0] == 1 :
            df_cleaned.loc[i,'model']=df_filter_dropdup.iloc[0]['model']
        elif df_filter.shape[0] == 0:
            df_cleaned.loc[i,'model']=df_cleaned.loc[i,'type']
        elif df_filter.shape[0] > 1:
            df_cleaned.loc[i,'model']=df_filter['model'].value_counts().index[0]


    # Check conflicting records between ‘type’ and ‘model’. If  ‘model’ value is not a subset of  ‘type’, classify the ‘model’ by its neighbours.  
    k=df_cleaned [(df_cleaned ['type'] !=df_cleaned ['model'])]#extract all rows going to process
    k1 = k.dropna(subset=['model'])#first ignore the empty value and do the easy task(combine executive condominium and walk-up apt)
    indices_model = k1.index
    for i in indices_model:
        condo_type,condo_model,condo_name=df_cleaned.loc[i]['type'],df_cleaned.loc[i]['model'],df_cleaned.loc[i]['name']
        if condo_type in condo_model:
            df_cleaned.loc[i,'type']=condo_model
        elif condo_model=='walk-up apt':
            df_cleaned.loc[i,'type']=condo_model
        elif condo_model not in condo_type_all:
            df_cleaned.loc[i,'model']=df_cleaned.loc[i,'type']
        else:
            df_filter = df_cleaned[df_cleaned['name'] == condo_name]
            df_filter_dropdup=df_filter[df_filter['type']==df_filter['model']].drop_duplicates('type')
            if df_filter_dropdup.shape[0] == 1 :
                df_cleaned.loc[i,'type']=df_filter_dropdup.iloc[0]['type']
                df_cleaned.loc[i,'model']=df_filter_dropdup.iloc[0]['type']
            elif df_filter_dropdup.shape[0]== 0:
                df_cleaned.loc[i,'model']=df_cleaned.loc[i,'type']
            elif df_filter_dropdup.shape[0]> 1:
                df_cleaned.loc[i,'model']=df_filter['model'].value_counts().index[0]
                df_cleaned.loc[i,'type']=df_cleaned.loc[i,'model']

    # Correct the potential noisy records in ‘model’ value via majority vote by ‘name’.
    indice_not=[]
    indices_model = df_cleaned.index
    for i in indices_model:
        condo_type,condo_model,condo_name=df_cleaned.loc[i]['type'],df_cleaned.loc[i]['model'],df_cleaned.loc[i]['name']
        df_name=df_cleaned[(df_cleaned['name']==condo_name)]
        df_namefilter=df_name[df_name['type']!=condo_type]
        if (df_namefilter.drop_duplicates('type').shape[0]>1):
            df_cleaned.loc[i,'type']=df_name['model'].value_counts().index[0]
            df_cleaned.loc[i,'model']=df_cleaned.loc[i,'type']
            indice_not.append(i)
    number=len(indice_not)
    return df_cleaned, number

In [3]:
def fill_median(df_in, target, relate, k):
    df = df_in.copy()
    indices_target = df.index[df[target].isna()]
    indices_length = indices_target.shape[0]
    if indices_length == 0:
        print('No missing value')
        return
    relate_r = df[~df[target].isna()][relate].to_numpy()
    target_r = df[~df[target].isna()][target].to_numpy()
    relate_r_t = np.tile(relate_r, (indices_length,1))
    target_r_t = np.tile(target_r, (indices_length,1))
    relate_w = df.loc[indices_target, relate].to_numpy()
    relate_w = relate_w.reshape(indices_length,1)
    diff = np.abs(relate_r_t-relate_w)
    k_nearest_indices = np.argsort(diff, axis = 1)[:,:k]
    k_targets = target_r_t[:, k_nearest_indices[:,:]]
    medians = np.ceil(np.median(k_targets[0,:,:], axis = 1)).astype(int)
    df.loc[indices_target, target] = medians
    return df

In [4]:
def fill_median_by_str(df_in, target, relate):
    df_cleaned = df_in.copy()
    indices_target_na = df_cleaned.index[df_cleaned[target].isna()]
    target_no_na = df_cleaned.dropna(subset=[target])
    medians, indices_keep, indices_drop = [], [], []
    for i in indices_target_na:
        condo_relate = df_cleaned.loc[i][relate]
        df_filter = target_no_na[(target_no_na[relate] == condo_relate)]
        if df_filter.shape[0] == 0:
            indices_drop.append(i)
            continue
        df_filter = df_filter[target].to_numpy()
        medians.append(np.median(df_filter))
        indices_keep.append(i)
    # df_cleaned = df_cleaned.drop(indices_drop, axis=0)
    df_cleaned.loc[indices_keep, target] = medians
    return df_cleaned, indices_drop

In [5]:
# calculate distance matrix for different auxiliary data
def location_attr(df, df_target):
    df_lat, df_lng = df['lat'].to_numpy(), df['lng'].to_numpy()
    df_target_lat, df_target_lng = df_target['lat'].to_numpy(), df_target['lng'].to_numpy()
    distances_mat = np.zeros((len(df), len(df_target)))
    for i in tqdm_notebook(range(len(df))):
        loc_house=(df_lat[i], df_lng[i])
        for j in range(len(df_target)):
            loc_target=(df_target_lat[j], df_target_lng[j])
            res = distance.distance(loc_house, loc_target).km
            distances_mat[i][j] = res
    df_distance = pd.DataFrame(distances_mat, columns=[df_target['name'].values])
    return distances_mat, df_distance

In [6]:
# count the number of target feature
def create_target_num(distances, lower, upper, name_str):
    target_num = np.zeros((distances.shape[0]))
    for r in range(distances.shape[0]):
        for c in range(distances.shape[1]):
            if lower<=distances[r][c]<=upper: target_num[r] += 1
    col_name = name_str + str(lower)+'to'+str(upper)
    df_target_num = pd.DataFrame(target_num, columns=[col_name])
    return df_target_num

In [7]:
def check_nan(df, k):
    total_na = df.isnull().sum().sort_values(ascending=False)
    percent = ((df.isnull().sum() / df.isnull().count()) * 100).sort_values(ascending=False)
    missing_data = pd.concat([total_na, percent], axis=1, keys=['Total', 'Percent'])
    missing_data = missing_data.reset_index()
    missing_data.columns = ['Name', 'Total', 'Percent']
    print(missing_data[:k])

In [9]:
def preprocess(df, train_set):
    df_cleaned = df.copy()
    if train_set:
        factors = ['name','street','type','model','bedrooms', 'bathrooms','district','region','tenure','built_year','no_of_units','area_size','price','date_listed']
        df_cleaned = df_cleaned.drop_duplicates(subset=factors)
        print('After removing duplicates, there are now {} records, each with {} attributes.'.format(df_cleaned.shape[0], df_cleaned.shape[1]))
    
    if train_set: 
        df_cleaned = df_cleaned.drop([2168,2415,4402,12891,13181,16294,19026,22378])
        print('After removing outliers, there are now {} records, each with {} attributes.'.format(df_cleaned.shape[0], df_cleaned.shape[1]))
    
    indices = df_cleaned['bedrooms'].str.contains('\+', na=False)
    rooms = df_cleaned.bedrooms[indices].to_numpy()
    f = lambda x: int(x[0]) + int(x[2])
    rooms = [f(r) for r in rooms]
    df_cleaned.loc[indices, 'bedrooms'] = rooms
    indices_notna = df_cleaned.bedrooms.notna()
    df_cleaned.loc[indices_notna, 'bedrooms'] = df_cleaned[df_cleaned.bedrooms.notna()].bedrooms.astype(str).astype(int)
    df_cleaned = fill_median(df_cleaned, 'bathrooms', 'area_size', 100)
    df_cleaned = fill_median(df_cleaned, 'bedrooms', 'area_size', 100)
    df_cleaned['bedrooms'] = df_cleaned.bedrooms.astype(str).astype(float)
    print('After filling bedrooms and bathrooms attributes, there are now {} records, each with {} attributes.'.format(df_cleaned.shape[0], df_cleaned.shape[1]))
    
    df_cleaned, miss_num = combine_type_model(df_cleaned, train_set)
    print('After combining model and type attributes, there are now {} records, each with {} attributes.'.format(df_cleaned.shape[0], df_cleaned.shape[1]))
    
    indices_tenure_na = df_cleaned['tenure'].isna()
    df_cleaned.loc[indices_tenure_na, 'tenure'] = 'leasehold'
    indices_freehold = df_cleaned['tenure'].str.contains('freehold|929|946|947|956|998|999|9999', regex=True)
    df_cleaned.loc[indices_freehold, 'tenure'] = 1
    df_cleaned.loc[~indices_freehold, 'tenure'] = 0
    df_cleaned['tenure'] = df_cleaned.tenure.astype(str).astype(int)
    print('After classifying tenure, there are now {} records, each with {} attributes.'.format(df_cleaned.shape[0], df_cleaned.shape[1]))
    
    df_cleaned, indice_drop = fill_median_by_str(df_cleaned, 'built_year', 'name')
    print('{} records that are unable to fill.'.format(len(indice_drop)))
    df_cleaned, indice_drop = fill_median_by_str(df_cleaned, 'built_year', 'street')
    print('{} records that are unable to fill.'.format(len(indice_drop)))
    df_cleaned, indice_drop1 = fill_median_by_str(df_cleaned, 'built_year', 'district')
    print('{} records that are unable to fill.'.format(len(indice_drop1)))
    print('After filling built_year attributes, there are now {} records, each with {} attributes.'.format(df_cleaned.shape[0], df_cleaned.shape[1]))
    
    df_cleaned, indice_drop = fill_median_by_str(df_cleaned, 'no_of_units', 'name')
    print('{} records that are unable to fill.'.format(len(indice_drop)))
    df_cleaned, indice_drop = fill_median_by_str(df_cleaned, 'no_of_units', 'street')
    print('{} records that are unable to fill.'.format(len(indice_drop)))
    df_cleaned, indice_drop1 = fill_median_by_str(df_cleaned, 'no_of_units', 'district')
    print('{} records that are unable to fill.'.format(len(indice_drop1)))
    print('After filling no_of_units attributes, there are now {} records, each with {} attributes.'.format(df_cleaned.shape[0], df_cleaned.shape[1]))
    
    irrelevants = ['listing_id','date_listed','type','market_segment','type_of_area','eco_category','accessibility']
    df_cleaned = df_cleaned.drop(columns=irrelevants)
    print('After dropping irrelevant features, there are now {} records, each with {} attributes.'.format(df_cleaned.shape[0], df_cleaned.shape[1]))
    
    print('Finally, there are now {} records, each with {} attributes.'.format(df_cleaned.shape[0], df_cleaned.shape[1]))
    return df_cleaned

In [10]:
# read train data + preprocess
df = pd.read_csv('train.csv', sep=',')
num_points, num_attributes = df.shape
print("There are {} data points, each with {} attributes.". format(num_points, num_attributes))
df_cleaned = preprocess(df, True)
check_nan(df_cleaned, 5)
df_cleaned.dtypes

There are 26048 data points, each with 23 attributes.
After removing duplicates, there are now 25722 records, each with 23 attributes.
After removing outliers, there are now 25714 records, each with 23 attributes.
After filling bedrooms and bathrooms attributes, there are now 25714 records, each with 23 attributes.
After combining model and type attributes, there are now 25714 records, each with 23 attributes.
After classifying tenure, there are now 25714 records, each with 23 attributes.
2642 records that are unable to fill.
1350 records that are unable to fill.
0 records that are unable to fill.
After filling built_year attributes, there are now 25714 records, each with 23 attributes.
1011 records that are unable to fill.
503 records that are unable to fill.
0 records that are unable to fill.
After filling no_of_units attributes, there are now 25714 records, each with 23 attributes.
After dropping irrelevant features, there are now 25714 records, each with 16 attributes.
Finally, the

name              object
street            object
model             object
bedrooms         float64
bathrooms        float64
district           int64
region            object
planning_area     object
subszone          object
lat              float64
lng              float64
tenure             int64
built_year       float64
no_of_units      float64
area_size        float64
price            float64
dtype: object

In [11]:
# Check counts for non-numerical features
check_attributes = ['model', 'region', 'tenure']
for att in check_attributes:
    print(df_cleaned[att].value_counts())

apartment                13708
condominium              11364
executive condominium      594
walk-up apt                 48
Name: model, dtype: int64
central region       17439
north-east region     2755
east region           2532
west region           2148
north region           840
Name: region, dtype: int64
0    15504
1    10210
Name: tenure, dtype: int64


In [12]:
df_cleaned.to_csv('data/train_data_cleaned.csv', index = False)

In [13]:
# read test data and preprocess
df_test = pd.read_csv('test.csv', sep=',')
num_points, num_attributes = df_test.shape
print("There are {} data points, each with {} attributes.". format(num_points, num_attributes))
noise_index = df_test[df_test['bedrooms']=="10+"].index
df_test.loc[noise_index[0],'bedrooms']=10
df_test_cleaned = preprocess(df_test, False)

There are 7500 data points, each with 22 attributes.
After filling bedrooms and bathrooms attributes, there are now 7500 records, each with 22 attributes.
After combining model and type attributes, there are now 7500 records, each with 22 attributes.
After classifying tenure, there are now 7500 records, each with 22 attributes.
1333 records that are unable to fill.
900 records that are unable to fill.
0 records that are unable to fill.
After filling built_year attributes, there are now 7500 records, each with 22 attributes.
312 records that are unable to fill.
159 records that are unable to fill.
0 records that are unable to fill.
After filling no_of_units attributes, there are now 7500 records, each with 22 attributes.
After dropping irrelevant features, there are now 7500 records, each with 15 attributes.
Finally, there are now 7500 records, each with 15 attributes.


In [14]:
# Check counts for non-numerical features
check_attributes = ['model', 'region', 'tenure']
for att in check_attributes:
    print(df_test_cleaned[att].value_counts())

apartment                3980
condominium              3331
executive condominium     169
walk-up apt                20
Name: model, dtype: int64
central region       5080
north-east region     801
east region           717
west region           647
north region          255
Name: region, dtype: int64
0    4439
1    3061
Name: tenure, dtype: int64


In [15]:
df_test_cleaned.to_csv('data/test_data_cleaned.csv', index = False)

In [16]:
# import all auxiliary features automatically
# 'pip install geopy' first to convert location
aux_files = ['sg-primary-schools','sg-secondary-schools','sg-train-stations','sg-gov-markets-hawker-centres','sg-shopping-malls','sg-commerical-centres']
target_names = ['primary_school_','sec_school_','train_','hawker_','shopping_','cc_']
df_cleaned = pd.read_csv('data/train_data_cleaned.csv', sep=',')
df_test_cleaned = pd.read_csv('data/test_data_cleaned.csv', sep=',')
for i in range(len(aux_files)):
    print(aux_files[i])
    df_target = pd.read_csv('auxiliary-data/auxiliary-data/'+aux_files[i]+'.csv', sep=',') 
    distances_mat, df_distance =location_attr(df_cleaned, df_target)
    df_distance.to_csv('dm_'+aux_files[i]+'.csv', index = False) 
    target_name = target_names[i]
    num_tar_01 = create_target_num(distances_mat, 0, 1, target_name)
    num_tar_01.to_csv(target_name+'01.csv', index = False)
    num_tar_02 = create_target_num(distances_mat, 0, 2, target_name)
    num_tar_02.to_csv(target_name+'02.csv', index = False)
    num_tar_03 = create_target_num(distances_mat, 0, 3, target_name)
    num_tar_03.to_csv(target_name+'03.csv', index = False)
    
    
    distances_mat_test, df_distance_test =location_attr(df_test_cleaned, df_target)
    df_distance_test.to_csv('dm_'+aux_files[i]+'_test.csv', index = False) 
    num_tar_01_test = create_target_num(distances_mat_test, 0, 1, target_name)
    num_tar_01_test.to_csv(target_name+'01_test.csv', index = False)
    num_tar_02_test = create_target_num(distances_mat_test, 0, 2, target_name)
    num_tar_02_test.to_csv(target_name+'02_test.csv', index = False)
    num_tar_03_test = create_target_num(distances_mat_test, 0, 3, target_name)
    num_tar_03_test.to_csv(target_name+'03_test.csv', index = False)

sg-primary-schools



sg-secondary-schools



sg-train-stations



sg-gov-markets-hawker-centres



sg-shopping-malls



sg-commerical-centres
